**<h1>Group Project</h1>**

**<h1>Lyrics to singer classification</h1>**
---

> 

   **<h2>Մասնակիցներ</h2>**


*   Հրանուշ Սահրադյան
*   Հասմիկ Մնացականյան
*   Մերի Վարդանյան
*   Արմեն Մանուկյան



**<h2>Purpose of the project</h2>**

Նախագծի նպատակն է համացանցից հավաքել (web scraping-ի միջոցով) 20 տարբեր հայ երգիչների երգերի բառեր, դրանք մշակելով ստանալ տվյալների բազմություն, բաժանել մի քանի մասի (train, val, test) և այդ տվյալների բազմության վրա ստանալ այնպիսի մոդել, որի միջոցով հնարավոր կլինի ինչ֊որ ճշտությամբ գուշակել, թե որ երգիչներին են պատկանում մոդելի կողմից չտեսնված երգերի բառերը։


[<h1>Data Collection</h1>](#collection)
In this notebook you can see how we have scraped data from various web pages.




During the data collection phase, we performed web scraping from multiple sources to gather the necessary information for our project. Specifically, we extracted data from websites such as [lilasar](http://lilasar.com/songs/),[genius](https://genius.com/) and [lyricstranslate](https://lyricstranslate.com/), etc . These websites were selected based on their relevance to our domain of interest.

Through the web scraping process, we targeted specific web pages that contained the desired data, including artist names, song titles, and accompanying text. By accessing the HTML structure of each page, we utilized various techniques and tools to extract the relevant information accurately and efficiently.

Web scraping allowed us to automate the extraction process and retrieve a large volume of data from these sources. We carefully crafted scripts or utilized web scraping libraries, such as BeautifulSoup or Scrapy, to navigate through the web pages, locate the relevant elements, and extract the required data fields.

The artist name, song title, and accompanying text were critical components of our dataset. The artist name provided us with valuable information about the creators of the songs, while the song title helped us identify and categorize each individual piece. The accompanying text contained lyrics, descriptions, or other textual information related to the songs, providing further context and enriching the dataset.

By leveraging web scraping techniques, we were able to efficiently collect the necessary data from multiple sources, ensuring a diverse and comprehensive dataset for our project. This data served as the foundation for subsequent steps in the model development process, including data cleaning, preprocessing, and ultimately, training and evaluating our models.




<a name="collection"></a>

## Data Collection

Import of libraries.

In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time 
import glob
import requests

**Web Scraping from [lilasar](http://lilasar.com/songs/s/) of 4 artists.**

In [11]:
# Send a GET request to the website
url = 'http://lilasar.com/songs/s/'  # Replace with the URL of the website you want to scrape
response = requests.get(url)

# Create a BeautifulSoup object to parse the HTML content
soup = BeautifulSoup(response.content,'html.parser')

In [13]:
div_elements = soup.find_all("div", class_="list-group")

In [14]:
artists_name=[]
song_title=[]
song_text=[]

for div_element in div_elements:
    artists = div_element.find_all("a")
    for artist in artists:
        #print(artist.text)
        if artist.text in ['ԱՐԹՈՒՐ ՄԵՍՉՅԱՆ','ՌՈՒԲԵՆ ՀԱԽՎԵՐԴՅԱՆ','ԿՈՄԻՏԱՍ','ՍԱՅԱԹ-ՆՈՎԱ']:
            name=artist.text
            new_url = artist.get("href")
            new_response = requests.get(new_url)
            new_soup = BeautifulSoup(new_response.content,'html.parser')
            poem_list=new_soup.find_all("ul")
            for poem in poem_list:
                song_urls=poem.find_all("li")
                for song_url in song_urls:
                    artists_name.append(name)
                    song_title.append(song_url.text)
                    song_response = requests.get(song_url.find("a").get("href"))
                    song_soup = BeautifulSoup(song_response.content,'html.parser')
                    song_texts=song_soup.find_all("div",class_="poem_div")
                    text=""
                    for sentence in song_texts:
                        text+=sentence.text
                    song_text.append(text)    

In [15]:
data = {"Artist": artists_name, "Song": song_title, "Lyrics":song_text}
df1 = pd.DataFrame(data)
df1 = df1[df1['Lyrics'] != '']
df1 

,Artist,Song,Lyrics
2,ԱՐԹՈՒՐ ՄԵՍՉՅԱՆ,ԱՀԱ ԵՎ ՎԵՐՋ,\r\n\tԱհա և վերջ...\n\r\n\tՎերջ հարաժամ և հարա...
3,ԱՐԹՈՒՐ ՄԵՍՉՅԱՆ,ԱՄԵՆԸ ՁԵԶ,\r\n\tՄեծածավալ և հիասքանչ տաճարների\n\r\n\tԵվ...
4,ԱՐԹՈՒՐ ՄԵՍՉՅԱՆ,ԱՆԱՌԱԿ ՈՐԴՈՒ ՎԵՐԱԴԱՐՁԸ,"\r\n\tԵրբ անառակ որդու նման,\n\r\n\tԴառնաս՝ թո..."
5,ԱՐԹՈՒՐ ՄԵՍՉՅԱՆ,ԱՇԽԱՐՀԻ ՑԱՎՈՎ,"\r\n\tՍիրտս կարյունի աշխարհի ցավով,\n\r\n\tՈրք..."
6,ԱՐԹՈՒՐ ՄԵՍՉՅԱՆ,ԱՌԱՋ ԳԻՏԵԻ,"\r\n\tԱռաջ գիտեի,\n\r\n\tԹե մարդիկ ինչու կգայի..."
...,...,...,...
219,ԿՈՄԻՏԱՍ,ՍԱՐԵՐԻ ՎՐՈՎ ԳՆԱՑ,"\r\n\tՍարերի վրով գնաց,\n\r\n\t Յա՜ր, յ..."
220,ԿՈՄԻՏԱՍ,"ԵՐԻ՛, ԵՐԻ՛, ԵՐԻ՛ ՋԱՆ","\r\n\tԷս գիշեր լուսը տեսա,\n\r\n\t Երի՛..."
221,ԿՈՄԻՏԱՍ,ՓԵՍԻՆ ԳՈՎՔԸ,"\r\n\tՄեր թագվորն էր խաչ,\n\r\n\tԽաչվառ խաչ ու..."
222,ԿՈՄԻՏԱՍ,"ՔԵԼԵՐ, ՑՈԼԵՐ","\r\n\tՔելեր, ցոլեր իմ յարը,\n\r\n\t Արև..."


In [17]:
df1 = df1[df1['Lyrics'] != '']

In [18]:
df1

,Artist,Song,Lyrics
2,ԱՐԹՈՒՐ ՄԵՍՉՅԱՆ,ԱՀԱ ԵՎ ՎԵՐՋ,\r\n\tԱհա և վերջ...\n\r\n\tՎերջ հարաժամ և հարա...
3,ԱՐԹՈՒՐ ՄԵՍՉՅԱՆ,ԱՄԵՆԸ ՁԵԶ,\r\n\tՄեծածավալ և հիասքանչ տաճարների\n\r\n\tԵվ...
4,ԱՐԹՈՒՐ ՄԵՍՉՅԱՆ,ԱՆԱՌԱԿ ՈՐԴՈՒ ՎԵՐԱԴԱՐՁԸ,"\r\n\tԵրբ անառակ որդու նման,\n\r\n\tԴառնաս՝ թո..."
5,ԱՐԹՈՒՐ ՄԵՍՉՅԱՆ,ԱՇԽԱՐՀԻ ՑԱՎՈՎ,"\r\n\tՍիրտս կարյունի աշխարհի ցավով,\n\r\n\tՈրք..."
6,ԱՐԹՈՒՐ ՄԵՍՉՅԱՆ,ԱՌԱՋ ԳԻՏԵԻ,"\r\n\tԱռաջ գիտեի,\n\r\n\tԹե մարդիկ ինչու կգայի..."
...,...,...,...
219,ԿՈՄԻՏԱՍ,ՍԱՐԵՐԻ ՎՐՈՎ ԳՆԱՑ,"\r\n\tՍարերի վրով գնաց,\n\r\n\t Յա՜ր, յ..."
220,ԿՈՄԻՏԱՍ,"ԵՐԻ՛, ԵՐԻ՛, ԵՐԻ՛ ՋԱՆ","\r\n\tԷս գիշեր լուսը տեսա,\n\r\n\t Երի՛..."
221,ԿՈՄԻՏԱՍ,ՓԵՍԻՆ ԳՈՎՔԸ,"\r\n\tՄեր թագվորն էր խաչ,\n\r\n\tԽաչվառ խաչ ու..."
222,ԿՈՄԻՏԱՍ,"ՔԵԼԵՐ, ՑՈԼԵՐ","\r\n\tՔելեր, ցոլեր իմ յարը,\n\r\n\t Արև..."


In [19]:
df1.to_csv('lilasar.csv', index=False)

**Web Scraping part from [lyricstranslate](https://lyricstranslate.com/en/country/armenia) of 19 artists.**


In [22]:
webdriver_path = 'chromedriver'
chrome_options = Options()
chrome_options.add_argument("--headless")  # To run Chrome in headless mode

driver = webdriver.Chrome(service=Service(webdriver_path), options=chrome_options)

driver.get('https://lyricstranslate.com/')
df_lyrics = pd.DataFrame(columns=['Artist', 'Song', 'Lyrics'])
songs_icon = driver.find_element(By.XPATH, '//a[@href="/en/songs" and contains(@class, "head-menu")]')
songs_icon.click()
artists =  ['HT Hayko','Empyray',' Lav Eli',' Armenchik',"Forsh", "Էմմի",'Ներսիկ Իսպիրյան',"Silva Hakobyan",'Անդրե','Լիլիթ Հովհաննիսյան', 'Սիրուշո', 'Միհրան Ծառուկյան','Arame','Razmik Amyan','Harout Pamboukjian','Հայկ Հակոբյան','Gor Sujyan','Arsen Safaryan','Aram MP3']



artist_search_box = driver.find_element(By.CSS_SELECTOR, 'input#edit-formtree-ltsearchtextartist')
for artist in artists:
    artist_search_box = driver.find_element(By.CSS_SELECTOR, 'input#edit-formtree-ltsearchtextartist')
    artist_search_box.clear()
    artist_search_box.send_keys(artist)
    artist_search_box.send_keys(Keys.RETURN)
    time.sleep(2) 
    song_links = driver.find_elements(By.CSS_SELECTOR, 'div.stt a[href^="/en/"]')

    
    for i in range(len(song_links)):
       
        song_links = driver.find_elements(By.CSS_SELECTOR, 'div.stt a[href^="/en/"]')
        song_link = song_links[i]
        song_url = song_link.get_attribute('href')
        song_name = song_link.text
        song_link.click()
        time.sleep(2)
        lyrics_element = driver.find_element(By.CSS_SELECTOR, 'div.ltf')
        lyrics = lyrics_element.text
        df_lyrics = df_lyrics.append({'Artist': artist, 'Song': song_name, 'Lyrics': lyrics}, ignore_index=True)
        driver.back()
        time.sleep(2)  
    print(f"Scraped songs for artist: {artist}")

C:\Users\User\AppData\Local\Temp\ipykernel_12652\509284065.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_lyrics = df_lyrics.append({'Artist': artist, 'Song': song_name, 'Lyrics': lyrics}, ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_12652\509284065.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_lyrics = df_lyrics.append({'Artist': artist, 'Song': song_name, 'Lyrics': lyrics}, ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_12652\509284065.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_lyrics = df_lyrics.append({'Artist': artist, 'Song': song_name, 'Lyrics': lyrics}, ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_12652\509284065.py:35: FutureWarning: T

Scraped songs for artist: HT Hayko


C:\Users\User\AppData\Local\Temp\ipykernel_12652\509284065.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_lyrics = df_lyrics.append({'Artist': artist, 'Song': song_name, 'Lyrics': lyrics}, ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_12652\509284065.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_lyrics = df_lyrics.append({'Artist': artist, 'Song': song_name, 'Lyrics': lyrics}, ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_12652\509284065.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_lyrics = df_lyrics.append({'Artist': artist, 'Song': song_name, 'Lyrics': lyrics}, ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_12652\509284065.py:35: FutureWarning: T

Scraped songs for artist: Empyray


C:\Users\User\AppData\Local\Temp\ipykernel_12652\509284065.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_lyrics = df_lyrics.append({'Artist': artist, 'Song': song_name, 'Lyrics': lyrics}, ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_12652\509284065.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_lyrics = df_lyrics.append({'Artist': artist, 'Song': song_name, 'Lyrics': lyrics}, ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_12652\509284065.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_lyrics = df_lyrics.append({'Artist': artist, 'Song': song_name, 'Lyrics': lyrics}, ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_12652\509284065.py:35: FutureWarning: T

...

C:\Users\User\AppData\Local\Temp\ipykernel_12652\509284065.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_lyrics = df_lyrics.append({'Artist': artist, 'Song': song_name, 'Lyrics': lyrics}, ignore_index=True)


Scraped songs for artist: Arsen Safaryan


C:\Users\User\AppData\Local\Temp\ipykernel_12652\509284065.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_lyrics = df_lyrics.append({'Artist': artist, 'Song': song_name, 'Lyrics': lyrics}, ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_12652\509284065.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_lyrics = df_lyrics.append({'Artist': artist, 'Song': song_name, 'Lyrics': lyrics}, ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_12652\509284065.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_lyrics = df_lyrics.append({'Artist': artist, 'Song': song_name, 'Lyrics': lyrics}, ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_12652\509284065.py:35: FutureWarning: T

C:\Users\User\AppData\Local\Temp\ipykernel_12652\509284065.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_lyrics = df_lyrics.append({'Artist': artist, 'Song': song_name, 'Lyrics': lyrics}, ignore_index=True)


Scraped songs for artist: Aram MP3


In [25]:
df_lyrics.to_csv('lyricstranslate.csv', index=False)

**Web Scraping part from [genius](https://genius.com/artists/Mi-qani-hogi) of 2 artists.**


In [27]:
# Set up Selenium options
chrome_options = Options()
chrome_options.add_argument("--headless")  # To run Chrome in headless mode

# Set up the path to your chromedriver executable
webdriver_path = 'chromedriver'

# Instantiate a webdriver instance
driver = webdriver.Chrome(service=Service(webdriver_path), options=chrome_options)

# Navigate to the web page
url = "https://genius.com/artists/Misho"
driver.get(url)
time.sleep(2)  # Allow 2 seconds for the web page to open

# Find the button to show all songs by Misho and simulate a click
button = driver.find_element(By.CSS_SELECTOR, 'body > routable-page > ng-outlet > routable-profile-page > ng-outlet > routed-page > profile-page > div.column_layout > div.column_layout-column_span.column_layout-column_span--primary > artist-songs-and-albums > div.full_width_button.u-clickable')
if button:
    button.click()

    # Wait for the page to load after the button click
    driver.implicitly_wait(5)  # Adjust the waiting time as needed



    # Find the scrollable element
    scroll_pause_time = 1 # You can set your own pause time. My laptop is a bit slow so I use 1 sec
    
    
    target=driver.find_element(By.CSS_SELECTOR,'body > div.modal_window > div.modal_window-content.modal_window-content--narrow_width.modal_window-content--white_background > ng-transclude > artist-songs > scrollable-data > div:nth-child(1) > transclude-injecting-local-scope:nth-child(20)')
    driver.execute_script('arguments[0].scrollIntoView(true);', target)
    
    target=driver.find_element(By.CSS_SELECTOR,'body > div.modal_window > div.modal_window-content.modal_window-content--narrow_width.modal_window-content--white_background > ng-transclude > artist-songs > scrollable-data > div:nth-child(1) > transclude-injecting-local-scope:nth-child(30)')
    driver.execute_script('arguments[0].scrollIntoView(true);', target)
    
    target=driver.find_element(By.CSS_SELECTOR,'body > div.modal_window > div.modal_window-content.modal_window-content--narrow_width.modal_window-content--white_background > ng-transclude > artist-songs > scrollable-data > div:nth-child(1) > transclude-injecting-local-scope:nth-child(40)')
    driver.execute_script('arguments[0].scrollIntoView(true);', target)
    
    target=driver.find_element(By.CSS_SELECTOR,'body > div.modal_window > div.modal_window-content.modal_window-content--narrow_width.modal_window-content--white_background > ng-transclude > artist-songs > scrollable-data > div:nth-child(1) > transclude-injecting-local-scope:nth-child(50)')
    driver.execute_script('arguments[0].scrollIntoView(true);', target)
    
    
    updated_html = driver.page_source

else:
    print("Button not found.")


In [28]:
lyrics=[]
songs=[]
artist=[]
# Use BeautifulSoup to parse the HTML content
soup = BeautifulSoup(updated_html, 'html.parser')
element = soup.select_one('body > div.modal_window > div.modal_window-content.modal_window-content--narrow_width.modal_window-content--white_background > ng-transclude > artist-songs > scrollable-data > div:nth-child(1)')
song_list = element.find_all("div", class_="profile_list_item")
for song in song_list:
    #artist.append("Misho")
    song_url = song.find("a").get("href")
    song_info=song.find("div",class_="mini_card-title_and_subtitle")
    info=[]
    for i in song_info:
        info.append(i.text)
    if info[3].strip()=="Misho":    
        songs.append(info[1])
        artist.append(info[3])
        song_page = requests.get(song_url)
        soup1 = BeautifulSoup(song_page.content, "html.parser")
        text_elements = soup1.find("div", class_="Lyrics__Container-sc-1ynbvzw-5 Dzxov")
        text=""
        for sent in text_elements:
            text=text+"\n"+sent.text
        lyrics.append(text)  
        
driver.quit()   

In [29]:
artist=[item.strip() for item in artist]

In [30]:
data = {"Artist": artist, "Song": songs, "Lyrics":lyrics}
df2 = pd.DataFrame(data)
df2.to_csv('misho.csv', index=False)

In [32]:
# Set up Selenium options
chrome_options = Options()
chrome_options.add_argument("--headless")  # To run Chrome in headless mode

# Set up the path to your chromedriver executable
webdriver_path = 'chromedriver'

# Instantiate a webdriver instance
driver = webdriver.Chrome(service=Service(webdriver_path), options=chrome_options)

# Navigate to the web page
url = "https://genius.com/artists/Mi-qani-hogi"
driver.get(url)
time.sleep(2)  # Allow 2 seconds for the web page to open

# Find the button to show all songs by Misho and simulate a click
button = driver.find_element(By.CSS_SELECTOR, 'body > routable-page > ng-outlet > routable-profile-page > ng-outlet > routed-page > profile-page > div.column_layout > div.column_layout-column_span.column_layout-column_span--primary > artist-songs-and-albums > div.full_width_button.u-clickable')
if button:
    button.click()

    # Wait for the page to load after the button click
    driver.implicitly_wait(2)  # Adjust the waiting time as needed    
else:
    print("Button not found.")

In [33]:
lyrics=[]
songs=[]
artist=[]
# Use BeautifulSoup to parse the HTML content
soup = BeautifulSoup(updated_html, 'html.parser')
element = soup.select_one('body > div.modal_window > div.modal_window-content.modal_window-content--narrow_width.modal_window-content--white_background > ng-transclude > artist-songs > scrollable-data > div')
song_list = element.find_all("div", class_="profile_list_item")
for song in song_list:
    song_url = song.find("a").get("href")
    song_info=song.find("div",class_="mini_card-title_and_subtitle")
    info=[]
    for i in song_info:
        info.append(i.text)
    songs.append(info[1])
    artist.append(info[3])
    song_page = requests.get(song_url)
    soup1 = BeautifulSoup(song_page.content, "html.parser")
    text_elements = soup1.find("div", class_="Lyrics__Container-sc-1ynbvzw-5 Dzxov")
    #print(text_elements)
    text=""
    for sent in text_elements:
        text=text+"\n"+sent.text
    lyrics.append(text)    

In [34]:
artist=[item.strip() for item in artist]
artist

['Mi Qani Hogi',
 'Mi Qani Hogi',
 'Misho',
 'Halfcastromeo',
 'Misho',
 'Mi Qani Hogi',
 'Vnasakar',
 'Misho',
 'Mi Qani Hogi',
 'Scimmie Del Deserto',
 'CKK',
 'Mi Qani Hogi',
 'CKK',
 'Mi Qani Hogi',
 'Mi Qani Hogi',
 'Mi Qani Hogi',
 'Mi Qani Hogi',
 'Misho',
 'Misho',
 'CKK',
 'Mi Qani Hogi',
 'Misho',
 'Mi Qani Hogi',
 'Misho',
 'Mi Qani Hogi',
 'JFK379',
 'Misho',
 'Misho',
 'Eresunmek',
 'Misho',
 'CKK',
 'Mente',
 'Misho',
 'CKK',
 'Majurakk, misho, mariel gesualdi',
 'Misho',
 'Misho',
 'Misho',
 'Misho',
 'Misho',
 'Misho',
 'Cris',
 'Misho',
 'Mi Qani Hogi',
 'Misho',
 'Misho',
 'Misho',
 'Enomenal',
 'Misho',
 'Misho',
 'Misho',
 'Misho',
 'Misho',
 'Misho',
 'Misho',
 'Misho',
 'Misho',
 'Misho',
 'Misho']

In [35]:
songs

['SMS',
 'Tar Indz',
 'Hastat Apeh',
 'OG',
 'Srtis Hyury',
 'Yeghanak',
 'Sax Qcel Em Texe',
 'Nax',
 'Axjik Chxjik',
 'Qui',
 'Hollywood is Dead',
 'Hayuhi Geghecik',
 'Biohazard',
 'Dzmer',
 'Nerqevic',
 'Ancoghika',
 'Barer Beranics',
 'Oves Du',
 'Zarancanq',
 'Goodbye California',
 'Chanapar',
 'Sagh Menak Uzumen',
 'Es Pahin',
 'M.I.Sh.O.-N Ed Hapavuma',
 'Toxer',
 'Day ’n night',
 'Tra Una Follia E L’Altra',
 '\u200bintrovert (call me crazy)',
 'Dus gam cxem',
 'Txur Mardik',
 'Intempo',
 'Un Giro',
 'Saghse Unenq',
 'Freddy vs. Jason',
 'Miss You',
 'Mets Tema',
 'Tsar',
 'Esor',
 'Hangistem Tanum',
 'Jamanaky',
 'Korela Kareli',
 'Vibe',
 'Menq',
 'Lav Jamanak',
 'Cinik',
 'Shog',
 'Orinakeli',
 '4-th Element (CYPHER)',
 '100 Tokos Mug',
 'Mi Te',
 'Cigan',
 'Sa',
 'Banaliz',
 'Yete Irar Lsenq',
 'I’m Lezun',
 'Chimanayir',
 'Biti Beat (Yerb Xpuma Beaty)',
 'Hachelia',
 '3 Mard']

In [36]:
data = {"Artist": artist, "Song": songs, "Lyrics":lyrics}
df3 = pd.DataFrame(data)
df3.to_csv('miqanihogi.csv', index=False)

 ####   Final Dataframe

In [37]:
df1=pd.read_csv("lilasar.csv")
df2=pd.read_csv("lyricstranslate.csv")
df3=pd.read_csv("misho.csv")
df4=pd.read_csv("miqanihogi.csv")

In [38]:
df = pd.concat([df1, df2,df3, df4], ignore_index=True)

In [39]:
df.to_csv('FinalData.csv', index=False)